In [ ]:
# @title
import numpy as np
import pandas as pd

# Day 20: Richardson Extrapolation and Derivatives by Interpolation

In this notebook we consider two additional techniques. The first, Richardson Extrapolation, is a general method which can be used to improve the accuracy of an approximation provided that the dominant error term takes the form $E\left(h\right) = ch^p$, where $c$ and $p$ are constants. The second additional technique uses *interpolation* methods to approximate the derivative of $f\left(x\right)$ in cases where only a discrete set of observed values satisfying $f\left(x\right)$ are known.

## Richardson Extrapolation

Richardson Extrapolation is a general class of techniques for increasing the accuracy of some numerical procedures -- including finite difference methods. The general strategy appears below.

Suppose that we have a strategy for approximating a quantity $G$ and that the resulting approximation for $G$ depends on a parameter $h$. We can write that approximation by $G = g\left(h\right) + E\left(h\right)$, where $E\left(h\right)$ is the approximation error at $h$. Richardson Extrapolation attempts to remove the error, provided that it has the form $E\left(h\right) = ch^p$ with $c$ and $p$ as constants. If this is the case, we have

\begin{align*}
G &= g\left(h_1\right) + E\left(h_1\right)\\
\implies G &= g\left(h_1\right) + ch_1^p
\end{align*}

We can do the same with several different values of the parameter $h$ to obtain the system:

$$\left\{\begin{array}{lcl} G & = & g\left(h_1\right) + ch_1^p\\
G & = & g\left(h_2\right) + ch_2^p
\end{array}\right.$$

We can multiply the top equation by $h_2^p$ and the bottom equation by $h_1^p$, and then subtract to eliminate $c$.

\begin{align*} G\cdot h_1^p - G\cdot h_2^p &= \left(h_1^p\cdot g\left(h_2\right) + ch_1^ph_2^p\right) - \left(h_2^p\cdot g\left(h_1\right) + ch_1^ph_2^p\right)\\
\implies G\cdot h_1^p - Gh_2^p &= h_1^p\cdot g\left(h_2\right) - h_2^p\cdot g\left(h_1\right)\\
G\left(h_1^p - h_2^p\right) &= h_1^p\cdot g\left(h_2\right) - h_2^p\cdot g\left(h_1\right)\\
\implies G &= \frac{h_1^p\cdot g\left(h_2\right) - h_2^p\cdot g\left(h_1\right)}{h_1^p - h_2^p}\\
\implies G &= \frac{h_2^p\left(\frac{h_1^p}{h_2^p}\cdot g\left(h_2\right) - g\left(h_1\right)\right)}{h_2^p\left(\frac{h_1^p}{h_2^p} - 1\right)}\\
\implies G &= \frac{\left(\frac{h_1}{h_2}\right)^p\cdot g\left(h_2\right) - g\left(h_1\right)}{\left(\frac{h_1}{h_2}\right)^p - 1}
\end{align*}

If we use $h_2 = h_1/2$, then the last line above becomes

$$G = \frac{2^p\cdot g\left(h_1/2\right) - g\left(h_1\right)}{2^p - 1}$$

Notice now that the error term has been eliminated! In the previous notebook, we examined the finite difference method to approximate $f''\left(x\right)$ for $f\left(x\right) = e^{-x}$ at $x = 1$. Let's do the same here, but utilize Richardson Extrapolation to improve our approximation. Note that since our truncation error when we did this in the previous notebook was $\mathscr{O}\left(h^2\right)$, we have $p = 2$.

**Example:** Use the first centered finite difference approximation in conjunction with Richardson Extrapolation to approximate $f'\left(x\right)$ for $f\left(x\right) = e^{-x}$ at $x = 1$.

In [ ]:
def f(x):
  return np.exp(-x)

h = np.array([0.64, 0.32, 0.16, 0.08, 0.04, 0.02, 0.01, 0.005, 0.0025, 0.00125])
x = 1.0

est_6d = np.round((np.round(f(x + h), 6) - 2*np.round(f(x), 6) + np.round(f(x - h), 6))/np.round(h**2, 6), 6)
est_8d = np.round((np.round(f(x + h), 8) - 2*np.round(f(x), 8) + np.round(f(x - h), 8))/np.round(h**2, 8), 8)

results_df = pd.DataFrame({"h": h, "Six-Digit Precision" : est_6d, "Eight-Digit Precision" : est_8d})

p = 2

results_df["Richardson Extrapolation (6-digit)"] = (2**p*results_df["Six-Digit Precision"].shift(-1) - results_df["Six-Digit Precision"])/(2**p - 1)
results_df["Richardson Extrapolation (8-digit)"] = (2**p*results_df["Eight-Digit Precision"].shift(-1) - results_df["Eight-Digit Precision"])/(2**p - 1)

results_df

If you remember, the value of $f''\left(1\right)$ for $f\left(x\right) = e^{-x}$ is near $0.367879$. We obtain an estimate accurate to four decimal places even for quite large $h$. At $h_1 = 0.64$, we are already as accurate as the most accurate eight-digit precision estimate even though we are only using six-digit precision in the fourth column. Notice that using smaller values of $h$ quickly lead to round-off error issues, regardless of the level of precision we are utilizing. This is something to remain aware of!

### More Examples

Let's see some additional examples which will enlighten us to additional methods for improving our estimates.

**Example 5.1:** Given the evenly spaced observed data points in the table below, compute $f'\left(x\right)$ and $f''\left(x\right)$ at $x = 0$ and $x = 0.2$ using finite difference approximations of $\mathscr{O}\left(h^2\right)$.

<center>

$x$ | $y$
--- | ---
0 | 0.0000
0.1 | 0.0819
0.2 | 0.1341
0.3 | 0.1646
0.4 | 0.1797


</center>

> *Solution.*

**Example 5.2:** Use the data in ***Example 5.1*** to estimate $f'\left(0\right)$ as accurately as possible.

> *Solution.*

**Example 5.3:** The linkage shown below has the dimensions $a = 100mm$, $b = 120mm$, $c = 150mm$, and $d = 180mm$.

![linkage](https://drive.google.com/uc?export=view&id=1wmLIOgrRcuYvRR3tQQp5Zoe9Pr5z_DCN)

Using geometry, we can show that the relationship between the angles $\alpha$ and $\beta$ is

<center>

$\alpha$ (degree) | $\beta$ (radians)
--- | ---
0 | 1.6595
5 | 1.5434
10 | 1.4186
15 | 1.2925
20 | 1.1712
25 | 1.0585
30 | 0.9561

</center>

If link $AB$ rotates with constant angular velocity $25~\text{rad/sec}$, use finite difference approximations of $\mathcal{O}\left(h^2\right)$ to tabulate the angular velocity $\frac{d\beta}{dt}$ of link $BC$ against $\alpha$.

> *Solution.*

## Derivatives by Interpolation

You may have noted that our finite difference techniques require that observed data points be equally spaced. This is perhaps an unrealistic expectation. In cases where observations are recorded along uneven intervals of $x$, then finite difference methods don't apply.

### Derivatives via Polynomial Interpolants

Recall that we know how to fit a polynomial interpolant of degree $n$ through $n+1$ observed data points. That is, we can fit

$$P_n\left(x\right) = a_0 + a_1x + a_2x^2 + \cdots + a_nx^n$$

Since we can fit this polynomial, we can compute and evaluate any of its derivatives at any desired $x$. As a reminder, we mentioned earlier that keeping the degree of the polynomial low is advisable to avoid our interpolant having wild oscillation between observed points. For this reason, we typically use *local interpolation* when our goal is to approximate a derivative. That is, if we wish to approximate $f^{\left(k\right)}\left(x_0\right)$, we'll build a polynomial interpolant using only the observed data points nearest $x_0$.

**Note:** If observed data points are equally spaced, then polynomial interpolation and finite difference methods will give the same results. Finite difference methods are a special case of polynomial interpolation to approximate derivatives.

In order to compute the derivative of our fitted polynomial, we'll need to obtain its coefficients. The only method we have that does this from earlier in our course is the *least squares fitting algorithm*. Recall that using a number of terms equal to the number of observed data points we have, however, results in fitting an interpolant. We can use this technique to approximate our derivative. If we know that our observed data contains noise, then we can fit a polynomial using least squares with a lesser number of terms and use that to approximate our derivatives.

### Derivatives via Polynomial and Cubic Spline Interpolants

We said earlier in our course that, because of its stiffness, a cubic spline is a good *global* interpolant. Additionally, since it is a piecewise cubic function, it is also easy to differentiate!

Recall that the second derivatives $k_i$ of the spline at the knot values are computed at the beginning of the process of constructing the spline. We did this with the `curvatures()` function from Day 9. From here, we can compute the first and second derivatives of the spline at any interior $x$ using:

\begin{align*} f_{i, i+1}'\left(x\right) &= \frac{k_i}{6}\left[\frac{3\left(x - x_i\right)^2}{x_i - x_{i+1}} - \left(x_i - x_{i+1}\right)\right] - \frac{k_{i+1}}{6}\left[\frac{3\left(x - x_i\right)^2}{x_i - x_{i+1}} - \left(x_i - x_{i+1}\right)\right] + \frac{y_i - y_{i+1}}{x_i - x_{i+1}}\\
f_{i, i+1}''\left(x\right) &= k_i\left(\frac{x - x_{i+1}}{x_i - x_{i+1}}\right) - k_{i+1}\left(\frac{x - x_i}{x_i - x_{i+1}}\right)
\end{align*}

What concerns might we have in estimating derivatives from polynomial interpolants?

Let's work through some examples.

**Example 5.4a:** Given the observed data below, compute $f'\left(2\right)$ and $f''\left(2\right)$ using a polynomial interpolant over three points.

<center>

x | f(x)
--- | ---
1.5 | 1.0628
1.9 | 1.3961
2.1 | 1.5432
2.4 | 1.7249
2.6 | 1.8423
3.1 | 2.0397

</center>

> *Solution.*

**Example 5.4b:** Given the observed data below, compute $f'\left(2\right)$ and $f''\left(2\right)$ using a cubic spline interpolant spanning all of the observed data points.

<center>

x | f(x)
--- | ---
1.5 | 1.0628
1.9 | 1.3961
2.1 | 1.5432
2.4 | 1.7249
2.6 | 1.8423
3.1 | 2.0397

</center>

> *Solution.*

**Example 5.5:** The observed data below is known to be *noisy*. Determine $f'\left(0\right)$ and $f'\left(1\right)$ as best as possible. Consider multiple fits, including quadratic through fourth-order.

<center>

x | f(x)
--- | ---
0 | 1.9934
0.2 | 2.1465
0.4 | 2.2129
0.6 | 2.1790
0.8 | 2.0683
1.0 | 1.9448
1.2 | 1.7655
1.4 | 1.5891

</center>

> *Solution.*